### SageMaker eNDPOINTS

## Falcon b instruct sagemaker endpoint

In [1]:
import json
import boto3

from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler

In [2]:


class ContentHandlerLLM(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({'inputs': prompt, 'parameters': model_kwargs})
        return input_str.encode('utf-8')
      
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))        
        return response_json[0]["generated_text"]

In [3]:
endpoint_name = 'falcon-7b-instruct'
aws_region = boto3.Session().region_name


In [4]:
parameters = {
    "max_new_tokens": 300,
    "return_full_text": False,
    "do_sample": True,
    "top_p": 0.3,
}
content_handler = ContentHandlerLLM()

llm = SagemakerEndpoint(
    endpoint_name = endpoint_name, 
    region_name = aws_region, 
    model_kwargs = parameters,
    content_handler = content_handler
)

In [5]:
llm("What is the smallest mammal")

'?\nThe smallest mammal is the bat, which can be as small as 2 inches in size.'

## Embedding Sagemaker Endpoint

In [9]:
from typing import Dict, List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
import json
import time

In [10]:
# extend the SagemakerEndpointEmbeddings class from langchain to provide a custom embedding function
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(
        self, texts: List[str], chunk_size: int = 5
    ) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        st = time.time()
        for i in range(0, len(texts), _chunk_size):
            response = self._embedding_func(texts[i:i + _chunk_size])
            results.extend(response)
        time_taken = time.time() - st
        print(f"got results for {len(texts)} in {time_taken}s, length of embeddings list is {len(results)}")
        return results

In [11]:



class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            inputs: List of input strings.
            model_kwargs: Additional keyword arguments to be passed to the endpoint.
        Returns:
            The transformed bytes input.
        """
        # Example: inference.py expects a JSON string with a "inputs" key:
        input_str = json.dumps({"inputs": inputs, "parameters": model_kwargs})  
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        # Example: inference.py returns a JSON string with the list of
        # embeddings in a "vectors" key:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["vectors"]




In [12]:
content_handler = ContentHandler()
emb_parameters = {"device": "cuda"}
embeddings = SagemakerEndpointEmbeddingsJumpStart(
    # credentials_profile_name="credentials-profile-name",
    endpoint_name="sentence-transformers-all-mpnet-base-v2",
    region_name=aws_region,
    content_handler=content_handler,
    model_kwargs=emb_parameters,
)


In [13]:
query_result = embeddings.embed_query("foo")
query_result

[0.0069312662817537785,
 0.07213316857814789,
 0.021989325061440468,
 -0.06384090334177017,
 -0.03433249890804291,
 0.02019808255136013,
 -0.002283054403960705,
 -0.010014386847615242,
 -0.025041036307811737,
 0.01987377367913723,
 0.01986796408891678,
 -0.02323724515736103,
 -0.01264805719256401,
 0.03335568308830261,
 0.046388085931539536,
 -0.0019061985658481717,
 -0.009752805344760418,
 0.001717908657155931,
 0.004696939140558243,
 0.012421111576259136,
 0.001757027581334114,
 0.010682749561965466,
 0.020206483080983162,
 0.015338404104113579,
 -0.02505332976579666,
 -0.03872108832001686,
 0.025440044701099396,
 0.02485748939216137,
 -0.054637882858514786,
 0.061127159744501114,
 0.021653439849615097,
 -0.02891891822218895,
 0.0029148580506443977,
 -0.009513716213405132,
 2.3598872758157086e-06,
 -0.026214109733700752,
 0.013212797231972218,
 0.006970277056097984,
 0.030711578205227852,
 -0.0114677082747221,
 -0.05772367864847183,
 0.0022857740987092257,
 -0.04081360995769501,
 0.0

In [14]:
len(query_result)

768

In [15]:
query_result[0]

0.0069312662817537785

In [16]:
doc_results = embeddings.embed_documents(["foo"])
doc_results

got results for 1 in 0.2436835765838623s, length of embeddings list is 1


[[0.0069312662817537785,
  0.07213316857814789,
  0.021989325061440468,
  -0.06384090334177017,
  -0.03433249890804291,
  0.02019808255136013,
  -0.002283054403960705,
  -0.010014386847615242,
  -0.025041036307811737,
  0.01987377367913723,
  0.01986796408891678,
  -0.02323724515736103,
  -0.01264805719256401,
  0.03335568308830261,
  0.046388085931539536,
  -0.0019061985658481717,
  -0.009752805344760418,
  0.001717908657155931,
  0.004696939140558243,
  0.012421111576259136,
  0.001757027581334114,
  0.010682749561965466,
  0.020206483080983162,
  0.015338404104113579,
  -0.02505332976579666,
  -0.03872108832001686,
  0.025440044701099396,
  0.02485748939216137,
  -0.054637882858514786,
  0.061127159744501114,
  0.021653439849615097,
  -0.02891891822218895,
  0.0029148580506443977,
  -0.009513716213405132,
  2.3598872758157086e-06,
  -0.026214109733700752,
  0.013212797231972218,
  0.006970277056097984,
  0.030711578205227852,
  -0.0114677082747221,
  -0.05772367864847183,
  0.002285

## Steps

![Steps](image.jpg)

### Load document

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from pprint import pprint
import os
from dotenv import load_dotenv

In [14]:
loader = DirectoryLoader("docs/css_lectures/", glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader)
pages = loader.load()

100%|██████████| 3/3 [00:00<00:00,  3.41it/s]


In [11]:
loader = PyPDFLoader("docs/css_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()
len(pages)

22

In [12]:
pages[0]

Document(page_content='MachineLearning-Lecture01  \nInstructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is ju st spend a little time going over the logistics \nof the class, and then we\'ll start to  talk a bit about machine learning.  \nBy way of introduction, my name\'s  Andrew Ng and I\'ll be instru ctor for this class. And so \nI personally work in machine learning, and I\' ve worked on it for about 15 years now, and \nI actually think that machine learning is th e most exciting field of all the computer \nsciences. So I\'m actually always excited about  teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thin g in computer science, but \nthe most exciting thing in all of human e ndeavor, so maybe a little bias there.  \nI also want to introduce the TAs, who are all graduate students doing research in or \nrelated to the machine learni ng and all aspects of machin e l

### Split the document

In [13]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [14]:
splits = text_splitter.split_documents(pages)
len(splits)

57

### Vectorstore

In [18]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chromahf/'



save Document

In [16]:
#do this when you load the documents at first to store vectors in Chromahf
vectordb = Chroma.from_documents(
    documents=splits, persist_directory=persist_directory, embedding=embeddings
)

print(vectordb._collection.count())


got results for 57 in 46.76255989074707s, length of embeddings list is 57
57


load document

In [19]:
# do this if you have already loaded the document and it is stored in Chromahf

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print(vectordb._collection.count())

57


In [61]:
vectordb.similarity_search("what is machine learning")

[Document(page_content='very excited about the new projector in this room, but I guess we\'ll see that in operation \non Wednesday.  \nSo start by talking about what machine learni ng is. What is machine learning? Actually, \ncan you read the text out there? Raise your hand if the text on the small screens is legible. \nOh, okay, cool, mostly legible. Okay. So I\'ll just read it out.  \nSo what is machine learning? Way back in  about 1959, Arthur Samuel defined machine \nlearning informally as the [inaudible] that gives computers to learn — [inaudible] that \ngives computers the ability to learn without  being explicitly programmed. So Arthur \nSamuel, so way back in the history of m achine learning, actually did something very \ncool, which was he wrote a checkers progr am, which would play games of checkers \nagainst itself.  \nAnd so because a computer can play thousands  of games against itself relatively quickly, \nArthur Samuel had his program play thousands  of games against its

### Retrieval

In [28]:
from langchain.docstore.document import Document

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


    

def get_metadate(docs: List[Document]) -> None:
    for doc in docs:
        print(doc.metadata)


#### MMR

In [46]:
question = "what is regression?"

docs = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=3)

In [47]:
print(docs)

[Document(page_content="okay?  \nSo as an overview of what we're going to do in this class, this class is sort of organized \ninto four major sections. We're gonna talk about four major topics in this class, the first \nof which is supervised learning. So le t me give you an example of that.  \nSo suppose you collect a data set of housing prices. And one of the TAs, Dan Ramage, \nactually collected a data set for me last week to use in the example later. But suppose that \nyou go to collect statistics about how much hous es cost in a certain geographic area. And \nDan, the TA, collected data from housing pr ices in Portland, Oregon. So what you can do \nis let's say plot the square footage of the house against the list price of  the house, right, so \nyou collect data on a bunch of houses. And let' s say you get a data set like this with \nhouses of different sizes that are li sted for different amounts of money.  \nNow, let's say that I'm trying to sell a hous e in the same area as Po

In [48]:
get_metadate(docs)

{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


#### working with metadata 

In [53]:
question = "what is regression on page 11?"
docs = vectordb.similarity_search(question, k=2, filter={"page": 11})


In [54]:
pretty_print_docs(docs)

Document 1:

okay?  
So as an overview of what we're going to do in this class, this class is sort of organized 
into four major sections. We're gonna talk about four major topics in this class, the first 
of which is supervised learning. So le t me give you an example of that.  
So suppose you collect a data set of housing prices. And one of the TAs, Dan Ramage, 
actually collected a data set for me last week to use in the example later. But suppose that 
you go to collect statistics about how much hous es cost in a certain geographic area. And 
Dan, the TA, collected data from housing pr ices in Portland, Oregon. So what you can do 
is let's say plot the square footage of the house against the list price of  the house, right, so 
you collect data on a bunch of houses. And let' s say you get a data set like this with 
houses of different sizes that are li sted for different amounts of money.  
Now, let's say that I'm trying to sell a hous e in the same area as Portland, Oregon as 
whe

In [55]:
get_metadate(docs)

{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


#### working with metadata using self-query retriever


In [56]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms import OpenAI

In [57]:
metadata_field_info = [
    # AttributeInfo(
    #     name="source",
    #     description="The lecture the chunk is from, should be one of `docs/css_lectures\\MachineLearning-Lecture01.pdf`,`docs/css_lectures\\MachineLearning-Lecture02.pdf`, `docs/css_lectures\\MachineLearning-Lecture03.pdf`",
    #     type="string",
    # ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [58]:
document_content_description = "Lecture notes"
base_llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    base_llm, vectordb, document_content_description, metadata_field_info, verbose=True
)


In [59]:
question = "what is regression on page 11"

In [60]:
docs = retriever.get_relevant_documents(question)

c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\langchain\chains\llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='regression' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='page', value=11) limit=None


In [61]:
pretty_print_docs(docs)

Document 1:

okay?  
So as an overview of what we're going to do in this class, this class is sort of organized 
into four major sections. We're gonna talk about four major topics in this class, the first 
of which is supervised learning. So le t me give you an example of that.  
So suppose you collect a data set of housing prices. And one of the TAs, Dan Ramage, 
actually collected a data set for me last week to use in the example later. But suppose that 
you go to collect statistics about how much hous es cost in a certain geographic area. And 
Dan, the TA, collected data from housing pr ices in Portland, Oregon. So what you can do 
is let's say plot the square footage of the house against the list price of  the house, right, so 
you collect data on a bunch of houses. And let' s say you get a data set like this with 
houses of different sizes that are li sted for different amounts of money.  
Now, let's say that I'm trying to sell a hous e in the same area as Portland, Oregon as 
whe

In [62]:
get_metadate(docs)

{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


In [63]:
question = "what do they say about regression between page 3 and page 20"
docs = retriever.get_relevant_documents(question)


query='regression' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='page', value=3), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='page', value=20)]) limit=None


In [64]:
get_metadate(docs)

{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 11, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 12, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


#### Compression

In [86]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [87]:
# Wrap our vectorstore
base_llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(base_llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [88]:
question = "what did they say about Matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)


c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\langchain\chains\llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try a

In [89]:
pretty_print_docs(compressed_docs)

Document 1:

"I know R and MATLAB, and I personally end up using MATLAB quite a bit more often for various reasons."
----------------------------------------------------------------------------------------------------
Document 2:

"And the student said, "Oh, it was the MATLAB." So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, and we'll actually have a short MATLAB tutorial in one of the discussion sections for those of you that don't know it."
----------------------------------------------------------------------------------------------------
Document 3:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 4:

"S

In [90]:
get_metadate(compressed_docs)

{'page': 9, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 7, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


In [91]:
docs = vectordb.similarity_search(question, k=4)
pretty_print_docs(docs)

Document 1:

So later this quarter, we'll use the discussion sections to talk about things like convex 
optimization, to talk a little bit about hidde n Markov models, which is a type of machine 
learning algorithm for modeling time series and a few other things, so  extensions to the 
materials that I'll be covering in the main  lectures. And attend ance at the discussion 
sections is optional, okay?  
So that was all I had from l ogistics. Before we move on to start talking a bit about 
machine learning, let me check what questions you have. Yeah?  
Student : [Inaudible] R or something like that?  
Instructor (Andrew Ng) : Oh, yeah, let's see, right. So our policy has been that you're 
welcome to use R, but I would strongly advi se against it, mainly because in the last 
problem set, we actually supply some code th at will run in Octave  but that would be 
somewhat painful for you to translate into R yourself. So for your other assignments, if 
you wanna submit a solution in R, that'

In [92]:
get_metadate(docs)

{'page': 9, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 7, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


#### Combining Compression and MMR

In [111]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [112]:
question = "what did they say about matlab?"
mmr_compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(mmr_compressed_docs)

c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\langchain\chains\llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try a

Document 1:

"I personally end up using MATLAB quite a bit more often for various reasons."


In [113]:
get_metadate(mmr_compressed_docs)

{'page': 9, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


#### Combining Compression and threshold score

In [106]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "similarity_score_threshold", search_kwargs ={"score_threshold": 0.2}),
)

In [107]:
question = "what did they say about matlab?"
trsh_compressed_docs = compression_retriever.get_relevant_documents(question)


c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\langchain\chains\llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try a

In [108]:
print(trsh_compressed_docs)

[Document(page_content='"I personally end up using MATLAB quite a bit more often for various reasons."', metadata={'page': 9, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}), Document(page_content='"And the student said, "Oh, it was the MATLAB." So for those of you that don\'t know MATLAB yet, I hope you do learn it. It\'s not hard, and we\'ll actually have a short MATLAB tutorial in one of the discussion sections for those of you that don\'t know it."', metadata={'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}), Document(page_content='"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."', metadata={'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}), Document(page_content='"So one more organizationa

In [109]:
pretty_print_docs(trsh_compressed_docs)

Document 1:

"I personally end up using MATLAB quite a bit more often for various reasons."
----------------------------------------------------------------------------------------------------
Document 2:

"And the student said, "Oh, it was the MATLAB." So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, and we'll actually have a short MATLAB tutorial in one of the discussion sections for those of you that don't know it."
----------------------------------------------------------------------------------------------------
Document 3:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 4:

"So one more organizational

In [110]:
get_metadate(trsh_compressed_docs)

{'page': 9, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 7, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


### Question answering

#### RetrievalQA Chain

In [21]:
from langchain.chains import RetrievalQA

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    return_source_documents=True,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2}),
)

qa_chain

RetrievalQA(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=SagemakerEndpoint(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<botocore.client.SageMakerRuntime object at 0x000001CCF2C92850>, endpoint_name='fal

In [24]:
question = "what did they say about matlab?"

In [25]:
result = qa_chain({"query": question})
result

{'query': 'what did they say about matlab?',
 'result': '\n\nThe instructor recommends using MATLAB for the term project.',
 'source_documents': [Document(page_content="So later this quarter, we'll use the discussion sections to talk about things like convex \noptimization, to talk a little bit about hidde n Markov models, which is a type of machine \nlearning algorithm for modeling time series and a few other things, so  extensions to the \nmaterials that I'll be covering in the main  lectures. And attend ance at the discussion \nsections is optional, okay?  \nSo that was all I had from l ogistics. Before we move on to start talking a bit about \nmachine learning, let me check what questions you have. Yeah?  \nStudent : [Inaudible] R or something like that?  \nInstructor (Andrew Ng) : Oh, yeah, let's see, right. So our policy has been that you're \nwelcome to use R, but I would strongly advi se against it, mainly because in the last \nproblem set, we actually supply some code th at wi

In [26]:
result["result"].strip('\n\n')

'The instructor recommends using MATLAB for the term project.'

In [29]:
get_metadate(result["source_documents"])

{'page': 9, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}
{'page': 2, 'source': 'docs/css_lectures/MachineLearning-Lecture01.pdf'}


In [30]:
pretty_print_docs(result["source_documents"])

Document 1:

So later this quarter, we'll use the discussion sections to talk about things like convex 
optimization, to talk a little bit about hidde n Markov models, which is a type of machine 
learning algorithm for modeling time series and a few other things, so  extensions to the 
materials that I'll be covering in the main  lectures. And attend ance at the discussion 
sections is optional, okay?  
So that was all I had from l ogistics. Before we move on to start talking a bit about 
machine learning, let me check what questions you have. Yeah?  
Student : [Inaudible] R or something like that?  
Instructor (Andrew Ng) : Oh, yeah, let's see, right. So our policy has been that you're 
welcome to use R, but I would strongly advi se against it, mainly because in the last 
problem set, we actually supply some code th at will run in Octave  but that would be 
somewhat painful for you to translate into R yourself. So for your other assignments, if 
you wanna submit a solution in R, that'

### Setting prompt format

In [37]:
from langchain.chains.retrieval_qa.base import BaseRetrievalQA

def get_qa_result(question: str, qa_chain: BaseRetrievalQA)->str:
    result = qa_chain({"query": question})
    return result["result"]
 

In [ ]:
qa_result = lambda question: qa_chain({"query": question})

### Prompt template

In [33]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [41]:
from langchain.chat_models import ChatOpenAI
chat_llm = ChatOpenAI(temperature=0)

In [42]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [43]:
get_qa_result("what is regression?", qa_chain)

'Regression is a type of supervised learning where the goal is to predict a continuous output variable based on input variables. Thanks for asking!'

In [44]:
get_qa_result("what did they say about matlab?", qa_chain)

'The instructor advised against using R for the problem set because the code provided would be difficult to translate. They mentioned that MATLAB is worth learning and that they personally use it more often. Thanks for asking!'

In [45]:
get_qa_result("what makes tea green tea?", qa_chain)

"I don't know. Thanks for asking!"

#### RetrievalQA Chaintypes

#### Mapreduce chain type

In [49]:
qa_chain_mr = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vectordb.as_retriever(search_type='mmr', search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type="map_reduce"
)

In [50]:
get_qa_result("what did they say about matlab?", qa_chain_mr)

'There is no mention of MATLAB in the provided text.'

#### Refine chain type

In [59]:
qa_chain_refine = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vectordb.as_retriever(search_type='mmr', search_kwargs={"k": 2}),
    chain_type="refine"
)

In [60]:
get_qa_result("what is probability?", qa_chain_refine)

'Probability is a mathematical concept that quantifies the likelihood or chance of an event occurring. It is a numerical value between 0 and 1, where 0 represents an impossible event and 1 represents a certain event. Probability is used to measure uncertainty and make predictions about the likelihood of different outcomes in various situations. It is a fundamental concept in statistics, mathematics, and decision-making, and plays a crucial role in fields such as finance, science, and engineering. By analyzing probabilities, we can assess risks, make informed decisions, and understand the likelihood of specific events happening.'

### Limitation

In [62]:
from langchain.chains.retrieval_qa.base import BaseRetrievalQA

def get_qa_result(question: str, qa_chain: BaseRetrievalQA)->str:
    result = qa_chain({"query": question})
    return result["result"]


from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

from langchain.chat_models import ChatOpenAI
chat_llm = ChatOpenAI(temperature=0)


# Run chain
qa_chain = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [65]:
get_qa_result("what are the topics covered?", qa_chain)

'The topics covered include statistics, algebra, machine learning, and extensions of the material taught in the main lectures. Thanks for asking!'

In [66]:
get_qa_result("which of these topics were mentioned the most?", qa_chain)

'The topics mentioned the most are discussion sections, extensions of material, homework problems, and honor code violations. Thanks for asking!'

In [67]:
get_qa_result("Tell me how often each of these topics were treated?", qa_chain)

'The frequency of each topic being treated is not mentioned in the given context. Thanks for asking!'

In [64]:
get_qa_result("what did they say about matlab?", qa_chain)

'The instructor advised against using R for the problem set because the code provided would be difficult to translate. They mentioned that MATLAB is worth learning and that they personally use it more often. Thanks for asking!'

### Memory

In [73]:

from pprint import pprint

In [85]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, 
    memory_key="chat_history")


In [86]:
pprint(memory)

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')


In [92]:
from langchain.chains import ConversationalRetrievalChain

# Run chain
qa_chain_conv= ConversationalRetrievalChain.from_llm(
    chat_llm,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 3}),
    # return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
    memory=memory
)

In [94]:
result = qa_chain_conv({"question": "what are the topics covered?"})
result

{'question': 'what are the topics covered?', 'chat_history': [HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context does not provide specific information about the topics covered in the course.', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context provides information that the course covers technical content, including math and equations, which are detailed in the lecture notes available on the course homepage. It also mentions that there will be discussion sections to go over statistics and algebra as refreshers, and later in the quarter, there will be discussion sections to cover extensions of the main lecture material. Thanks for asking!', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The course covers topics 

In [96]:
result["question"]

'what are the topics covered?'

In [95]:
result['answer']

'The course covers topics in machine learning, including statistics and algebra as refreshers, extensions to the main lecture material, and the use of MATLAB or Octave for implementing learning algorithms. Thanks for asking!'

In [97]:
pprint(memory)

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context does not provide specific information about the topics covered in the course.', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context provides information that the course covers technical content, including math and equations, which are detailed in the lecture notes available on the course homepage. It also mentions that there will be discussion sections to go over statistics and algebra as refreshers, and later in the quarter, there will be discussion sections to cover extensions of the main lecture material. Thanks for asking!', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The course covers top

In [98]:
memory.chat_memory

ChatMessageHistory(messages=[HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context does not provide specific information about the topics covered in the course.', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context provides information that the course covers technical content, including math and equations, which are detailed in the lecture notes available on the course homepage. It also mentions that there will be discussion sections to go over statistics and algebra as refreshers, and later in the quarter, there will be discussion sections to cover extensions of the main lecture material. Thanks for asking!', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The course covers topics in machine learning, including st

In [99]:
memory.chat_memory.messages

[HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context does not provide specific information about the topics covered in the course.', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context provides information that the course covers technical content, including math and equations, which are detailed in the lecture notes available on the course homepage. It also mentions that there will be discussion sections to go over statistics and algebra as refreshers, and later in the quarter, there will be discussion sections to cover extensions of the main lecture material. Thanks for asking!', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The course covers topics in machine learning, including statistics and algebra as refr

In [100]:
memory.chat_memory.messages[0]

HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False)

In [103]:
result = qa_chain_conv({"question": "tell me more avbout these topics?"})
result

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-OKEaxoW73Lo9pocIXi1wDApB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


{'question': 'tell me more avbout these topics?', 'chat_history': [HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context does not provide specific information about the topics covered in the course.', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context provides information that the course covers technical content, including math and equations, which are detailed in the lecture notes available on the course homepage. It also mentions that there will be discussion sections to go over statistics and algebra as refreshers, and later in the quarter, there will be discussion sections to cover extensions of the main lecture material. Thanks for asking!', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The course covers to

In [102]:
memory.chat_memory.messages

[HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context does not provide specific information about the topics covered in the course.', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The context provides information that the course covers technical content, including math and equations, which are detailed in the lecture notes available on the course homepage. It also mentions that there will be discussion sections to go over statistics and algebra as refreshers, and later in the quarter, there will be discussion sections to cover extensions of the main lecture material. Thanks for asking!', additional_kwargs={}, example=False), HumanMessage(content='what are the topics covered?', additional_kwargs={}, example=False), AIMessage(content='The course covers topics in machine learning, including statistics and algebra as refr

### Chatbot gradio

In [119]:
def pretty_print_docs(docs: List[Document]) -> None:
    return f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)])


In [120]:
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document 


from dotenv import load_dotenv
load_dotenv()
# COHERE_API_KEY = os.getenv("COHERE_API_KEY")
cohere_embeddings = CohereEmbeddings()

cohere_embeddings.embed_query('Veronica is a Pythonista')

[3.5390625, -0.22058105, 0.8256836, 0.5131836, -2.1269531, 0.2376709, -0.67822266, 0.7680664, -0.41503906, -0.042633057, -0.120910645, -0.76904297, 0.18664551, -0.6635742, 0.828125, 1.1015625, 1.2236328, 1.7880859, 1.2724609, -0.87158203, -0.76708984, 2.0996094, 0.8129883, -0.87841797, 1.1787109, -0.97265625, 1.5478516, 1.0498047, -1.5712891, 1.4453125, 0.97021484, 0.2685547, 0.08001709, 1.2353516, -0.72265625, -0.86328125, 0.4724121, 0.44921875, 0.113342285, 0.5239258, 1.0849609, 1.6708984, -0.43774414, 1.0654297, 0.03955078, 1.2216797, -1.3388672, 0.44262695, 0.29663086, 2.0605469, 2.59375, 0.46362305, -2.7910156, 1.8740234, -0.19689941, -1.9873047, -2.9121094, -2.0605469, -0.84228516, -3.09375, -0.9042969, 2.3085938, -0.8833008, 1.3427734, -0.9057617, 1.2548828, -0.96435547, -2.09375, 1.1601562, 0.66552734, -0.027664185, -1.5898438, 0.9902344, -1.5537109, 0.6147461, 0.28588867, -1.15625, 0.14941406, 1.8007812, 0.17297363, 0.29956055, 1.0517578, 0.84033203, -1.7138672, -0.15161133, 0

In [132]:
def load_db(file):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = CohereEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # similarity search
    result = db.similarity_search("veronica is a pythonista")
    return pretty_print_docs(result)

In [133]:
load_db("docs/css_lectures/MachineLearning-Lecture01.pdf")

"Document 1:\n\non the project and Peter Renfrew and Mike and a few others. But I guess this really is a \ngood dog/bad dog since it's a robot dog.  \nThe second video on the right, some of the st udents, I guess Peter, Zico, Tonca working \non a robotic snake, again using learning algorith ms to teach a snake robot to climb over \nobstacles.  \nBelow that, this is kind of a fun example.  Ashutosh Saxena and Jeff Michaels used \nlearning algorithms to teach a car how to  drive at reasonably high speeds off roads \navoiding obstacles.  \nAnd on the lower right, that's a robot program med by PhD student Eva Roshen to teach a \nsort of somewhat strangely configured robot how to get on top of an obstacle, how to get \nover an obstacle. Sorry. I know the video's kind of small. I hope you can sort of see it. \nOkay?  \nSo I think all of these are robots that I thi nk are very difficult to hand-code a controller \nfor by learning these sorts of l earning algorithms. You can in relatively shor

In [134]:
import gradio as gr
import openai

def similarity_search(pdf_file):
    # Your existing code for loading and processing PDFs here
    result = load_db("docs/css_lectures/MachineLearning-Lecture01.pdf")
     
    return result

# Create a Gradio interface
iface = gr.Interface(
    fn=similarity_search,
    inputs=gr.inputs.File(type="file", label="Upload a PDF"),
    outputs=[gr.outputs.Textbox(label="Similarity Search Results")]
)

# Start the Gradio interface
iface.launch()

C:\Users\ve797\AppData\Local\Temp\ipykernel_37268\400466230.py:13: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.File(type="file", label="Upload a PDF"),
C:\Users\ve797\AppData\Local\Temp\ipykernel_37268\400466230.py:13: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.File(type="file", label="Upload a PDF"),
C:\Users\ve797\AppData\Local\Temp\ipykernel_37268\400466230.py:13: GradioDeprecationWarning: `keep_filename` parameter is deprecated, and it has no effect
  inputs=gr.inputs.File(type="file", label="Upload a PDF"),
C:\Users\ve797\AppData\Local\Temp\ipykernel_37268\400466230.py:14: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=[gr.outputs.Textbox(label="Similarity Search Results")]

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


## Chatbot

In [4]:
from scripts.sentence_transformer import(
    ContentHandler,
    SagemakerEndpointEmbeddingsJumpStart,
)

import boto3

aws_region = boto3.Session().region_name

content_handler = ContentHandler()
emb_parameters = {"device": "cuda"}



In [5]:
print(embeddings.embed_query("foo"))

[0.0069312662817537785, 0.07213316857814789, 0.021989325061440468, -0.06384090334177017, -0.03433249890804291, 0.02019808255136013, -0.002283054403960705, -0.010014386847615242, -0.025041036307811737, 0.01987377367913723, 0.01986796408891678, -0.02323724515736103, -0.01264805719256401, 0.03335568308830261, 0.046388085931539536, -0.0019061985658481717, -0.009752805344760418, 0.001717908657155931, 0.004696939140558243, 0.012421111576259136, 0.001757027581334114, 0.010682749561965466, 0.020206483080983162, 0.015338404104113579, -0.02505332976579666, -0.03872108832001686, 0.025440044701099396, 0.02485748939216137, -0.054637882858514786, 0.061127159744501114, 0.021653439849615097, -0.02891891822218895, 0.0029148580506443977, -0.009513716213405132, 2.3598872758157086e-06, -0.026214109733700752, 0.013212797231972218, 0.006970277056097984, 0.030711578205227852, -0.0114677082747221, -0.05772367864847183, 0.0022857740987092257, -0.04081360995769501, 0.01569967530667782, -0.004089626017957926, -0

In [9]:

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.base import BaseConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from typing import Literal

In [7]:
llm_name = "gpt-3.5-turbo"

In [10]:
def load_db(file: str, chain_type: Literal["stuff", "map_reduce"],  k: int) -> BaseConversationalRetrievalChain:
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = SagemakerEndpointEmbeddingsJumpStart(
    # credentials_profile_name="credentials-profile-name",
    endpoint_name="sentence-transformers-all-mpnet-base-v2",
    region_name=aws_region,
    content_handler=content_handler,
    model_kwargs=emb_parameters,
)
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 


In [14]:
qa = load_db("docs/css_lectures/MachineLearning-Lecture01.pdf", "stuff", 2)


got results for 77 in 28.408884048461914s, length of embeddings list is 77


In [15]:
result = qa({"question": "what is regression?", "chat_history": []})
result

{'question': 'what is regression?',
 'chat_history': [],
 'answer': 'Regression is a type of supervised learning problem where the goal is to predict a continuous value or variable, such as price. It involves analyzing a set of features or properties of a dataset and finding a mathematical relationship or pattern that can be used to make predictions. In regression, the focus is on estimating the value of the dependent variable based on the values of the independent variables.',
 'source_documents': [Document(page_content="regression problem. And the term regression sort of refers to the fact that the variable \nyou're trying to predict is a continuous value and price.  \nThere's another class of supervised learning problems which we'll talk about, which are \nclassification problems. And so, in a classifi cation problem, the variab le you're trying to \npredict is discreet rather than continuous . So as one specific example — so actually a \nstandard data set you can download online [i

In [17]:
result['answer']

'Regression is a type of supervised learning problem where the goal is to predict a continuous value or variable, such as price. It involves analyzing a set of features or properties of a dataset and finding a mathematical relationship or pattern that can be used to make predictions. In regression, the focus is on estimating the value of the dependent variable based on the values of the independent variables.'

In [18]:
result['source_documents']

[Document(page_content="regression problem. And the term regression sort of refers to the fact that the variable \nyou're trying to predict is a continuous value and price.  \nThere's another class of supervised learning problems which we'll talk about, which are \nclassification problems. And so, in a classifi cation problem, the variab le you're trying to \npredict is discreet rather than continuous . So as one specific example — so actually a \nstandard data set you can download online [i naudible] that lots of machine learning \npeople have played with. Let's say you collect  a data set on breast cancer tumors, and you \nwant to learn the algorithm to predict wh ether or not a certai n tumor is malignant. \nMalignant is the opposite of benign, right, so ma lignancy is a sort of harmful, bad tumor. \nSo we collect some number of features, some  number of properties of these tumors, and \nfor the sake of sort of having a simple [inaudi ble] explanation, let's just say that we're", me

In [19]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "docs/css_lectures/MachineLearning-Lecture01.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [20]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

# jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    # pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

got results for 77 in 36.01592516899109s, length of embeddings list is 77


Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()